# LAB 9 : Algoritmo Forward Backward en HMM

In [21]:
# definir las variables a utilizar
import random

# posibles estados
states = ['sunny', 'rainy']

# observaciones previas, iniciamos con unas 20
observations = []
for _ in range(0,20):
    observations.append(random.choice(states))
print(f"Observations : {observations}")

# probabilidades iniciales
sunny_prob = random.uniform(0,1)
rainy_prob = 1 - sunny_prob
initial_prob = {'sunny':sunny_prob , 'rainy': rainy_prob}
print(f"initial probability : {initial_prob}")

# probabilidades de transiciones
sunny_first = random.uniform(0,1)
rainy_first = random.uniform(0,1)
sunny_second = 1 - rainy_first
rainy_second = 1 - sunny_first
sunny_transition = {'sunny' : sunny_first, 'rainy' : rainy_second}
rainy_transition = {'sunny' : sunny_second, 'rainy' : rainy_first}
transition_prob = {'sunny': sunny_transition, 'rainy': rainy_transition}
print(f"transition probability : {transition_prob}")

# probabilidades de emisiones
# se actualizan las variables anteriores 
sunny_first = random.uniform(0,1)
rainy_first = random.uniform(0,1)
sunny_second = 1 - rainy_first
rainy_second = 1 - sunny_first
sunny_transition = {'sunny' : sunny_first, 'rainy' : rainy_second}
rainy_transition = {'sunny' : sunny_second, 'rainy' : rainy_first}
emission_prob = {'sunny': sunny_transition, 'rainy': rainy_transition}
print(f"emission probability : {emission_prob}")

Observations : ['rainy', 'sunny', 'sunny', 'rainy', 'rainy', 'rainy', 'sunny', 'rainy', 'sunny', 'sunny', 'rainy', 'rainy', 'sunny', 'sunny', 'sunny', 'rainy', 'sunny', 'sunny', 'sunny', 'rainy']
initial probability : {'sunny': 0.5746194754024322, 'rainy': 0.4253805245975678}
transition probability : {'sunny': {'sunny': 0.7602335406930292, 'rainy': 0.23976645930697082}, 'rainy': {'sunny': 0.36371329256453566, 'rainy': 0.6362867074354643}}
emission probability : {'sunny': {'sunny': 0.10679825082841488, 'rainy': 0.8932017491715851}, 'rainy': {'sunny': 0.21001422905913314, 'rainy': 0.7899857709408669}}


## Creacion del Forward Backward y HMM

In [41]:
class HMM:
    def __init__(self, states=None, observations=None, initial_prob=None, transition_prob=None, emission_prob=None):
        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob
    

    def forward(self, observations):
        alpha = [{}]
        # Base case for the Forward step
        for state in self.states:
            alpha[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        # Recursive case for the Forward step
        for t in range(1, len(observations)):
            alpha.append({})
            for current_state in self.states:
                alpha[t][current_state] = sum(
                    alpha[t-1][previous_state] * self.transition_prob[previous_state][current_state] *
                    self.emission_prob[current_state][observations[t]] for previous_state in self.states
                )
        return alpha

    def backward(self, observations):
        beta = [{} for _ in range(len(observations))]
        # Base case for the Backward step
        for state in self.states:
            beta[len(observations)-1][state] = 1

        # Recursive case for the Backward step
        for t in range(len(self.observations) - 2, -1, -1):
            for state in self.states:
                beta[t][state] = sum(
                    beta[t+1][future_state] * self.transition_prob[state][future_state] *
                    self.emission_prob[future_state][observations[t+1]] for future_state in self.states
                )
        return beta

    def generate_sequence(self, length=10):
        # Start with an initial state based on initial probabilities
        states_sequence = [random.choices(self.states, weights=self.initial_prob.values())[0]]
        observations_sequence = []

        # Generate the states sequence
        for i in range(1, length):
            # Get the last state's transition probabilities
            last_state = states_sequence[-1]
            current_state = random.choices(
                self.states,
                weights=[self.transition_prob[last_state][next_state] for next_state in self.states]
            )[0]
            states_sequence.append(current_state)

        # Generate the observations sequence based on states sequence
        for state in states_sequence:
            observation = random.choices(
                self.observations,
                weights=[self.emission_prob[state][obs] for obs in self.observations]
            )[0]
            observations_sequence.append(observation)

        return observations_sequence

    def compute_state_probabilities(self):
        pass


hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)
observations_hmm = hmm.generate_sequence()
print(f"Observations with HMM : {observations_hmm}")

forward = hmm.forward(observations)
print(f"forward : {forward}")

backward = hmm.backward(observations)
print(f"Backward : {backward}")



Observations with HMM : ['sunny', 'rainy', 'sunny', 'rainy', 'rainy', 'rainy', 'rainy', 'rainy', 'rainy', 'sunny']
forward : [{'sunny': 0.5132511205375111, 'rainy': 0.33604456166744}, {'sunny': 0.05472498197539796, 'rainy': 0.07074982273730579}, {'sunny': 0.0071914116367736355, 'rainy': 0.012209888508804287}, {'sunny': 0.008849888557849527, 'rainy': 0.007499531679759306}, {'sunny': 0.008445813367266873, 'rainy': 0.00544597133233199}, {'sunny': 0.007504290557226647, 'rainy': 0.004337197207858301}, {'sunny': 0.0007777592930814261, 'rainy': 0.0009574502725237943}, {'sunny': 0.0008391774169927755, 'rainy': 0.0006285865219632537}, {'sunny': 9.255094685961239e-05, 'rainy': 0.00012625380179705296}, {'sunny': 1.2418556138852359e-05, 'rainy': 2.1531546846258096e-05}, {'sunny': 1.5427659916945924e-05, 'rainy': 1.3175217015029082e-05}, {'sunny': 1.4756252115688753e-05, 'rainy': 9.544806250794045e-06}, {'sunny': 1.5688418160052137e-06, 'rainy': 2.0185071691983716e-06}, {'sunny': 2.0578357492201847